## Пошаговое руководство по созданию простейшего pipeline из Sci-kit Learn

#### Большинство проектов по науке о данных требуют определенного уровня очистки и предварительной обработки данных, чтобы максимально использовать модели машинного обучения. Вот некоторые распространенные предварительные обработки или преобразования:

а. Заполнение пропущенных значений

б. Удаление выбросов

в. Нормализация или стандартизация числовых признаков

д. Кодирование категориальных признаков

В научном наборе есть набор функций, поддерживающих такого рода преобразование, таких как StandardScaler, SimpleImputer и т. д., в пакете предварительной обработки.

Типичный и упрощенный рабочий процесс науки о данных примерно выглядит так:

-Получить данные для тренировки

-Очистить/предварительно обработать/преобразовать данные

-Обучить модель машинного обучения

-Оценить и оптимизировать модель

-Очистить/предварительно обработать/преобразовать новые данные

-Дать модели новые данным, чтобы делать прогнозы.

Вы можете заметить, что предварительная обработка данных должна выполняться как минимум дважды в рабочем процессе. Каким бы утомительным и трудоемким ни был этот шаг, было бы здорово, если бы мы могли автоматизировать этот процесс и применить его ко всем будущим новым наборам данных.

С конвейером обучения scikit мы можем легко систематизировать процесс и, следовательно, сделать его максимально воспроизводимым. Далее я проведу вас через процесс использования конвейера обучения scikit, чтобы облегчить вашу жизнь.

#### Чтение данных
Мы будем использовать данные «ежедневного проката велосипедов» из фантастического учебного материала Microsoft по машинному обучению .

In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/data/daily-bike-share.csv')
data.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
rentals         int64
dtype: object

In [3]:
data.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
rentals       0
dtype: int64

 К счастью, в этом наборе данных нет пропущенных значений. Хотя кажется, что все функции являются числовыми, на самом деле есть некоторые категориальные функции, которые нам необходимо идентифицировать.

In [4]:
data

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,rentals
0,1,1/1/2011,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331
1,2,1/2/2011,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131
2,3,1/3/2011,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120
3,4,1/4/2011,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108
4,5,1/5/2011,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,12/27/2012,1,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,247
727,728,12/28/2012,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,644
728,729,12/29/2012,1,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,159
729,730,12/30/2012,1,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,364


Давайте сначала отфильтруем некоторые явно бесполезные функции.

In [5]:
data = data[['season'
             , 'mnth'
             , 'holiday'
             , 'weekday'
             , 'workingday'
             , 'weathersit'
             , 'temp'
             , 'atemp'
             , 'hum'
             , 'windspeed'
             , 'rentals']]

In [6]:
data

,season,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,rentals
0,1,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331
1,1,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131
2,1,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120
3,1,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108
4,1,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82
...,...,...,...,...,...,...,...,...,...,...,...
726,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,247
727,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,644
728,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,159
729,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,364


#### Разделим данные
Перед созданием конвейера нам нужно сначала разделить данные на обучающий набор и тестовый набор.

In [7]:
from sklearn.model_selection import train_test_split
X = data.drop('rentals',axis=1)
y = data['rentals']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

#### Создадим конвейер (pipeline)
Основным параметром пайплайна, над которым мы будем работать, являются « шаги ». Легче просто взглянуть на то, как должен выглядеть конвейер:

In [ ]:
#Pipeline(steps=[('name_of_preprocessor', препроцессор), 
                #('name_of_ml_model', ml_model())])

Импортируем нужные библиотеки

In [8]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

Во-первых, нам нужно определить преобразователи как для числовых, так и для категориальных признаков. Шаг преобразования представлен кортежем. В этом кортеже вы сначала определяете имя преобразователя, а затем функцию, которую хотите применить. Порядок кортежа будет порядком, в котором конвейер применяет преобразования. Здесь мы сначала имеем дело с пропущенными значениями, затем стандартизируем числовые признаки и кодируем категориальные признаки.

In [9]:
numeric_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean'))
      ,('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='constant'))
      ,('encoder', OrdinalEncoder())
])

#### Preprocessor
Следующее, что нам нужно сделать, это указать, какие столбцы являются числовыми, а какие категориальными, чтобы мы могли соответствующим образом применить преобразователи. Мы применяем преобразователи к функциям с помощью ColumnTransformer. Применение преобразователей к функциям — это наш препроцессор. Подобно конвейеру, мы передаем список кортежей, который состоит из («имя», «трансформер», «функции») , параметру « трансформеры ».

In [10]:
numeric_features = ['temp', 'atemp', 'hum', 'windspeed']
categorical_features = ['season', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']
preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, numeric_features)
   ,('categorical', categorical_transformer, categorical_features)
])

In [ ]:
Можно создать список числовых/категориальных функций на основе типа данных, например:

In [12]:
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns
categorical_features = data.select_dtypes(include=['object']).columns

Я лично не рекомендую это, потому что если у вас есть категориальные признаки, замаскированные под числовой тип данных, например, этот набор данных, вы не сможете их идентифицировать. Используйте этот метод только в том случае, если вы на 100 % уверены, что только числовые признаки являются числовыми типами данных.

In [13]:
categorical_features

Index([], dtype='object')

#### Оценщик Estimator

После сборки нашего препроцессора мы можем добавить оценщик, который представляет собой алгоритм машинного обучения, который вы хотели бы применить, чтобы завершить наш конвейер предварительной обработки и обучения. Поскольку в этом случае целевая переменная непрерывна, я применю здесь модель случайной лесной регрессии.

In [14]:
from sklearn.ensemble import RandomForestRegressor
pipeline = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('regressor',RandomForestRegressor())
           ])

Чтобы создать модель, аналогичную тому, что мы делали с алгоритмом машинного обучения, мы используем функцию конвейера fit

In [15]:
rf_model = pipeline.fit(X_train, y_train)
print (rf_model)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'atemp', 'hum',
                                                   'windspeed']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('encoder',
                                                                   OrdinalEncoder())]),
                        

Используем обычные методы для оценки модели.

In [16]:
from sklearn.metrics import r2_score
predictions = rf_model.predict(X_test)
print (r2_score(y_test, predictions))

0.7683677302805014


#### Использование модели
Чтобы максимизировать воспроизводимость, мы хотели бы неоднократно использовать эту модель для наших новых входящих данных. Давайте сохраним модель, используя пакет joblib, чтобы сохранить ее

In [18]:
import joblib
joblib.dump(rf_model, './rf_model.pkl')

['./rf_model.pkl']

Теперь мы можем вызвать этот конвейер, который включает в себя все виды предварительной обработки данных, которые нам нужны, и модель обучения, когда нам это нужно.

In [21]:
rf_model = joblib.load('PATH/TO/rf_model.pkl')
new_prediction = rf_model.predict(new_data)

NameError: name 'new_data' is not defined

#### Заключение
До знакомства с конвейером обучения scikit мне всегда приходилось переделывать всю предварительную обработку и преобразование данных всякий раз, когда я хотел применить одну и ту же модель к разным наборам данных. Это был действительно утомительный процесс. Я попытался написать функцию, выполняющую их все, но результат меня не очень удовлетворил и не избавил меня от большого количества работы.

Конвейер обучения Scikit действительно делает мои рабочие процессы более плавными и гибкими. Например, вы можете легко сравнить производительность ряда алгоритмов, таких как:

In [ ]:
regressors = [
    regressor_1()
   ,regressor_2()
   ,regressor_3()
   ....]
for regressor in regressors:
    pipeline = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('regressor',regressor)
           ])
    model = pipeline.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print (regressor)
    print (f('Model r2 score:{r2_score(predictions, y_test)}')